<a href="https://colab.research.google.com/github/Crazy-Explorer31/News-assistant/blob/main/news_clf_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
df = pd.read_csv("articles.csv")

In [ ]:
df.head(2)

,Unnamed: 0,docid,title,modified,lastmodtime,type,domain,status,part,bloc,pubdate,text,rightcol,snippet,topic,len
0,1,1365127,Патриарх Кирилл назвал условие мира на Украине,1673048590,1673048590,1,1,0,0,1,1673048590,Патриарх Московский и всея Руси Кирилл Фото: K...,Патриарх Кирилл назвал условие мира на Украине,Патриарх Московский и всея Руси Кирилл ... и в...,0,1022
1,6,1365122,В McDonald's предупредили о сокращениях персонала,1673045801,1673045801,1,1,0,0,4,1673045801,Фото: Pavel Mikheyev / Reuters Марина Совина Г...,В McDonald's предупредили о сокращениях персонала,Фото: Pavel Mikheyev / Reuters Марина ... сокр...,1,1357


In [ ]:
from sklearn.model_selection import train_test_split

X = df[['text']]
y = df['topic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head(3)

,text
1861,Фото: Alexander Ermochenko / Reuters Варвара К...
91808,Фото: Michael Sohn / Reuters Платон Щукин Герм...
74802,Фото: Анна Майорова / «Лента.ру» Алевтина Запо...


In [ ]:
X_train.shape, X_test.shape

((122764, 1), (30692, 1))

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

russian_stopwords = stopwords.words("russian")

'''
vec = CountVectorizer(
    stop_words=russian_stopwords,
    ngram_range=(1, 2),
    min_df=3,
    max_df=0.85,
    max_features=70_000,
    lowercase=False
) # подбор гиперпараметров очень помогает
vec.fit(X_train['text'])
'''

tf_df_train = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=70_000,
    min_df=4,
    max_df=0.85,
    stop_words=russian_stopwords
)
tf_df_train.fit(X_train['text'])


TfidfVectorizer(max_df=0.85, max_features=70000, min_df=4, ngram_range=(1, 2),
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...])

In [ ]:
bow = tf_df_train.transform(X_train['text'])  # bow — bag of words (мешок слов)
bow_test = tf_df_train.transform(X_test['text'])
print(bow.shape)

(122764, 70000)


In [ ]:
scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

In [ ]:
# !!!! Этот GridSearch работает очень долго (около 2 часов).
# Результат его работы - {'C': 10, 'penalty': 'l2', 'solver': 'saga'}
# Я запускал его в блокноте с парсером, когда делал "сырое" решение.

from sklearn.model_selection import GridSearchCV

log_reg = LogisticRegression(max_iter=500, random_state=42)

param_grid = {
    'C': [0.1, 10, 100],
    'penalty': ['l2', 'elasticnet', 'none'],
    'solver': ['lbfgs', 'liblinear', 'saga']
}

grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(bow, y_train)

best_params = grid_search.best_params_
print("Лучшие параметры:", best_params)

clf = grid_search.best_estimator_
y_pred = clf.predict(bow_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовой выборке:", accuracy)

In [ ]:
# Метрики модели из предыдущей ячейки из "сырого" решения
y_pred = clf.predict(bow_test)
accuracy = accuracy_score(y_test, y_pred)
print("Точность на тестовой выборке:", accuracy)
train_pred = clf.predict(bow)
test_pred = clf.predict(bow_test)

train_acc = accuracy_score(y_train, train_pred)
test_acc = accuracy_score(y_test, test_pred)

print('Train accuracy:', round(train_acc, 3))
print('Test accuracy: ', round(test_acc, 3))

pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

tf_df_final = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=70_000,
    min_df=4,
    max_df=0.85,
    stop_words=russian_stopwords
)
tf_df_final.fit(X['text'])

TfidfVectorizer(max_df=0.85, max_features=70000, min_df=4, ngram_range=(1, 2),
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...])

In [ ]:
Test = pd.read_csv("test_news.csv")
Test

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [ ]:
bow = tf_df_final.transform(X['text'])
bow_test = tf_df_final.transform(Test['content'])

scaler = MaxAbsScaler()
bow = scaler.fit_transform(bow)
bow_test = scaler.transform(bow_test)

In [ ]:
final_clf = LogisticRegression(
    C=10,
    penalty="l2",
    solver="saga",
    max_iter=10_000,
    random_state=42
)
final_clf.fit(bow, y)
pred = final_clf.predict(bow_test)

In [ ]:
subm = pd.read_csv("base_submission_news.csv")
subm.head()

,topic,index
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


In [ ]:
subm['topic'] = pred
# subm.to_csv("result_csv.csv", index=False)
subm

,topic,index
0,0,0
1,1,1
2,4,2
3,1,3
4,4,4
...,...,...
26270,3,26270
26271,3,26271
26272,3,26272
26273,3,26273
